In [ ]:
!pip install beautifulsoup4

In [ ]:
!pip install lxml

In [ ]:
!pip install html5lib

In [ ]:
!pip install requests

In [ ]:
from bs4 import  BeautifulSoup
import requests
from urllib.request import urlopen
import pandas as pd

--> Scrape the Wikipedia page

In [ ]:
html = urlopen("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(html, 'lxml')
print(soup.prettify())

--> Copy data from the table in the Wikipedia page into a DataFrame

In [ ]:
table = soup.find('table')
#print(table)
table_col = table.find_all('th')
table_rows = table.find_all('tr')
#print(table_rows)

df = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        df.append(row)

df = pd.DataFrame(df, columns = ["Postcode", "Borough", "Neighbourhood"])
print(df)

--> Ignore cells with a borough that is Not assigned

In [ ]:
df_2=df[df.Borough.str.contains("Not assigned") == False]
print(df_2)

--> Combine rows having the same Postcode into one row with the neighborhoods separated with a comma

In [ ]:
df_3=df_2.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
print(df_3)

--> If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [ ]:
for index, row in df_3.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood']=row['Borough']
        print(row)

-->  Use the .shape method to print the number of rows of your dataframe

In [ ]:
df_3.shape

In [ ]:
geo_cord=pd.read_csv("http://cocl.us/Geospatial_data")
print(geo_cord.head(5))

In [ ]:
df_4=df_3.merge(geo_cord, left_on='Postcode', right_on='Postal Code', how='outer')
df_4.drop(["Postal Code"], axis=1, inplace=True)
print(df_4.head(5))